<a href="https://colab.research.google.com/github/ankit-rathi/Quantvesting_v2/blob/main/myXH_Return_Analysis.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [3]:
import yfinance as yf
import pandas as pd

def calculate_stock_gains(start_date, end_date, stock_list):
    """
    Calculate percentage gains for a list of NSE stocks over a specified date range.

    Args:
        start_date (str): Start date in 'YYYY-MM-DD' format.
        end_date (str): End date in 'YYYY-MM-DD' format.
        stock_list (list): List of NSE stock symbols (e.g., ['RELIANCE.NS', 'TCS.NS']).

    Returns:
        pd.DataFrame: DataFrame containing stock symbols and their percentage gains.
    """
    data = []

    for stock in stock_list:
        try:
            # Fetch stock data for the given date range
            stock_data = yf.Ticker(stock + '.NS').history(period='max', interval='1d')[map(str.title, ['open', 'close', 'low', 'high', 'volume'])]

            stock_data = stock_data[(stock_data.index <= end_date) & (stock_data.index >= start_date)]

            if not stock_data.empty:
                # Calculate percentage gain
                start_price = stock_data['Close'].iloc[0]
                end_price = stock_data['Close'].iloc[-1]
                percentage_gain = ((end_price - start_price) / start_price) * 100

                data.append({
                    'Symbol': stock,
                    'Start Price': start_price,
                    'End Price': end_price,
                    'Gains (%)': round(percentage_gain, 2)
                })
            else:
                data.append({
                    'Symbol': stock,
                    'Start Price': None,
                    'End Price': None,
                    'Gains (%)': None
                })
        except Exception as e:
            data.append({
                'Symbol': stock,
                'Start Price': None,
                'End Price': None,
                'Gains (%)': None,
                'Error': str(e)
            })

    # Create and return DataFrame
    return pd.DataFrame(data)

# function to fetch my prospects csv
def get_mypps_df():
  mypps_df = pd.read_csv('https://raw.githubusercontent.com/ankit-rathi/Tradevesting_v1/main/data/myProspectsScrips.csv')
  return mypps_df

# function to fetch screener data
def get_myscreen_df():
  myscreen_df = pd.read_csv('https://raw.githubusercontent.com/ankit-rathi/Tradevesting_v1/main/data/myScreenerDB.csv')
  return myscreen_df

# Example usage:
if __name__ == "__main__":
    start_date = "2025-03-04"
    end_date = "2025-07-19"


    pps_df = get_mypps_df()
    scr_df = get_myscreen_df()

    df = pd.merge(pps_df, scr_df, on = 'Symbol')

    #stock_list = df[(df['Conviction'].isin(['X','H'])) & ((df['CapType'] == 'LC'))]['Symbol'].values

    list_x40 = list(pps_df[pps_df['MBQ'].str.contains('X40', na=False)]['Symbol'].values)
    #list_x5k = list(pps_df[pps_df['MBQ'].str.contains('X5K', na=False)]['Symbol'].values)
    list_xh = list(df[(df['Conviction'].isin(['X','H'])) & ((df['CapType'] == 'LC'))]['Symbol'].values)
    list_x40n = list(pps_df[pps_df['MBQ'].str.contains('X40N', na=False)]['Symbol'].values)
    #len(set(list_x40 + list_x5k) - set( list_x40n))
    stock_list = list(set(list_x40 + list_xh) - set( list_x40n))

    #stock_list = ["RELIANCE", "TCS", "INFY", "HDFCBANK"]

    result_df = calculate_stock_gains(start_date, end_date, stock_list)
    result_df = pd.merge(result_df, df, on = 'Symbol')
    result_df['Conviction'] = result_df['Conviction'] + '-'+ result_df['CapType']
    cols = ['Symbol', 'Conviction', 'MBQ', 'InFolio', 'Start Price', 'End Price', 'Gains (%)']
    result_df = result_df[cols]
    print(str(round(result_df['Gains (%)'].mean())) + '%')

result_df.sort_values(by = 'Gains (%)', ascending=False)

16%


,Symbol,Conviction,MBQ,InFolio,Start Price,End Price,Gains (%)
17,NAM-INDIA,M-MC,X40,NaN,512.787231,841.450012,64.09
7,HDFCAMC,H-LC,X40,NaN,3626.935303,5590.000000,54.12
22,WHIRLPOOL,M-SC,XR/OX40,1.0,937.500000,1444.699951,54.10
29,GILLETTE,H-SC,X40,NaN,7783.100098,11258.000000,44.65
42,PFIZER,H-SC,X40,NaN,3962.305420,5319.500000,34.25
...,...,...,...,...,...,...,...
60,PGHH,H-MC,XY25/X40,1.0,13446.000000,13308.000000,-1.03
28,VOLTAS,H-MC,XY25/X40/AR,1.0,1401.725464,1377.000000,-1.76
34,INFY,H-LC,X40/X5K,1.0,1664.875122,1586.099976,-4.73
55,TCS,H-LC,X40/X5K,1.0,3489.091797,3189.899902,-8.58


In [ ]:
result_df.sort_values(by = 'Gains (%)', ascending=False).tail(20)